# Recommendation System

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv


In [2]:
movies_metadata = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")
movies_metadata.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# **By Popularity and genre**

**IMDb weighted rating formula: weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C**

Where:

R = average for the movie (mean) = (rating)

v = number of votes for the movie = (votes)

m = minimum votes required to be listed in the Top Rated list

C = the mean vote across the whole report

In [3]:
#Genre
genre = "Animation"
filter_genre = movies_metadata.copy()
filter_genre = filter_genre[filter_genre.genres.str.contains(genre,case=False)]
filter_genre.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
12,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",NaN,21032,tt0112453,en,Balto,An outcast half-wolf risks his life to prevent...,...,1995-12-22,11348324.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Part Dog. Part Wolf. All Hero.,Balto,False,7.1,423.0
47,False,"{'id': 136214, 'name': 'Pocahontas Collection'...",55000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,10530,tt0114148,en,Pocahontas,History comes gloriously to life in Disney's e...,...,1995-06-14,346079773.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An American legend comes to life.,Pocahontas,False,6.7,1509.0
235,False,"{'id': 410261, 'name': 'A Goofy Movie Collecti...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",NaN,15789,tt0113198,en,A Goofy Movie,"Though Goofy always means well, his amiable cl...",...,1995-04-07,35348597.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's the story of a father who couldn't be clo...,A Goofy Movie,False,6.7,404.0
240,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 16, 'na...",NaN,43475,tt0113234,en,Gumby: The Movie,The band is back together! Gumby reunites with...,...,1995-12-01,0.0,77.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The original green hero!,Gumby: The Movie,False,5.0,2.0


In [4]:
#Popularity - IMDb weighted formula
v = movies_metadata['vote_count']
R = movies_metadata['vote_average']
m = 25000
C = movies_metadata['vote_average'].mean()

filter_genre['imdb_formula'] =  (v / (v+m)) * R + (m / (v+m)) * C
filter_popularity = filter_genre.sort_values(by=["imdb_formula"], ascending=False)
filter_popularity[:20]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,imdb_formula
30315,False,NaN,175000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",http://movies.disney.com/inside-out,150540,tt2096673,en,Inside Out,"Growing up can be a bumpy road, and it's no ex...",...,8.576112e+08,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Meet the little voices inside your head.,Inside Out,False,7.9,6737.0,6.102577
13724,False,NaN,175000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://disney.go.com/disneypictures/up/,14160,tt1049413,en,Up,Carl Fredricksen spent his entire life dreamin...,...,7.350991e+08,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Up,False,7.8,7048.0,6.098027
12704,False,NaN,180000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://disney.go.com/disneypictures/wall-e/,10681,tt0910970,en,WALL·E,WALL·E is the last robot left on an Earth that...,...,5.213119e+08,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An adventure beyond the ordinar-E.,WALL·E,False,7.8,6439.0,6.065059
24455,False,NaN,165000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",http://movies.disney.com/big-hero-6,177572,tt2245084,en,Big Hero 6,The special bond that develops between plus-si...,...,6.521054e+08,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,False,7.8,6289.0,6.056741
359,False,"{'id': 94032, 'name': 'The Lion King Collectio...",45000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://movies.disney.com/the-lion-king,8587,tt0110357,en,The Lion King,A young lion cub named Simba can't wait to be ...,...,7.882418e+08,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Life's greatest adventure is finding your plac...,The Lion King,False,8.0,5520.0,6.048990
6232,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://movies.disney.com/finding-nemo,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",...,9.403355e+08,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7.6,6292.0,6.016694
4756,False,"{'id': 137696, 'name': 'Monsters, Inc. Collect...",115000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://movies.disney.com/monsters-inc,585,tt0198781,en,"Monsters, Inc.","James Sullivan and Mike Wazowski are monsters,...",...,5.628163e+08,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,We Scare Because We Care.,"Monsters, Inc.",False,7.5,6150.0,5.989733
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,3.735540e+08,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,5.988844
5481,False,NaN,15000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",http://movies.disney.com/spirited-away,129,tt0245429,ja,千と千尋の神隠し,A ten year old girl who wanders away from her ...,...,2.749251e+08,125.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,The tunnel led Chihiro to a mysterious town...,Spirited Away,False,8.3,3968.0,5.985556
36253,False,NaN,150000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://movies.disney.com/zootopia,269149,tt2948356,en,Zootopia,"Determined to prove herself, Officer Judy Hopp...",...,1.023784e+09,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Welcome to the urban jungle.,Zootopia,False,7.7,4961.0,5.962914


# **Content-Based Filtering**
Based on features: 
- Genre
- Keywords
- Cast (The first three)
- Crew (Only the director)

Features Overview and Tagline seem not really relevant

In [5]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [6]:
contentdf = movies_metadata.copy()
contentdf["genres"] = contentdf['genres'].fillna('[]').apply(literal_eval)\
                        .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).apply(' '.join)

contentdf = contentdf.drop(["adult", "belongs_to_collection", "budget", "homepage", "original_language", 
                "original_title", "popularity", "poster_path", 'production_companies',
                'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages',
                'status', 'video', 'vote_average', 'vote_count', "imdb_id", "overview", "tagline"], axis=1)

keywords = pd.read_csv("/kaggle/input/the-movies-dataset/keywords.csv")
keywords['keywords'] = keywords['keywords'].fillna('[]').apply(literal_eval)\
                        .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).apply(' '.join)

credits = pd.read_csv("/kaggle/input/the-movies-dataset/credits.csv")
credits['cast'] = credits['cast'].fillna('[]').apply(literal_eval)\
                    .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])\
                    .apply(lambda x: x[:3] if len(x) >=3 else x).apply('_'.join)
credits['crew'] = credits['crew'].fillna('[]').apply(literal_eval).apply(get_director)

Strange ids for movie with index: 19730, 29503, 35587

In [7]:
contentdf = contentdf.drop([19730, 29503, 35587])

In [8]:
contentdf['id'] = contentdf['id'].astype(int)
contentdf = contentdf.merge(keywords, on="id").merge(credits, on="id")

In [9]:
contentdf["words"] = contentdf["genres"].fillna(' ').str.lower() + " " + contentdf["keywords"].fillna(' ').str.lower() + " " + \
                        contentdf["cast"].fillna(' ').str.lower().str.replace(" ", "").str.replace("_"," ") + " " + \
                        contentdf["crew"].fillna(' ').str.lower().str.replace(" ", "")
contentdf = contentdf.drop(["genres","keywords", "cast", "crew"], axis=1)
contentdf.head()

,id,title,words
0,862,Toy Story,animation comedy family jealousy toy boy frien...
1,8844,Jumanji,adventure fantasy family board game disappeara...
2,15602,Grumpier Old Men,romance comedy fishing best friend duringcredi...
3,31357,Waiting to Exhale,comedy drama romance based on novel interracia...
4,11862,Father of the Bride Part II,comedy baby midlife crisis confidence aging da...


DROP duplicate titles and missing values

In [10]:
contentdf = contentdf.dropna().drop_duplicates(subset=['title'])

In [11]:
contentdf["words"] = pd.DataFrame(contentdf["words"].str.split(), columns=['words'])
lens = [len(item) for item in contentdf['words']]
contentdf = pd.DataFrame({"id" : np.repeat(contentdf['id'].values,lens),\
                              "title" : np.repeat(contentdf['title'].values,lens),\
                              "words" : np.hstack(contentdf['words']) })
contentdf.head()

,id,title,words
0,862,Toy Story,animation
1,862,Toy Story,comedy
2,862,Toy Story,family
3,862,Toy Story,jealousy
4,862,Toy Story,toy


In [12]:
contentdf["words"].value_counts()

drama                  18870
comedy                 13580
thriller                6985
romance                 6492
action                  6192
                       ...  
birding                    1
petercoggan                1
archiekao                  1
sanke                      1
fernandanegripouget        1
Name: words, Length: 72631, dtype: int64

Remove unique keywords

In [13]:
contentdf = contentdf[contentdf.duplicated(subset=['words'], keep=False)]
contentdf["words"].value_counts()

drama               18870
comedy              13580
thriller             6985
romance              6492
action               6192
                    ...  
glenmeadows             2
burtbalaban             2
grace                   2
laurencenaismith        2
vladimirsokoloff        2
Name: words, Length: 29861, dtype: int64

In [14]:
contentdf = contentdf.groupby('title').agg(words=('words', ' '.join)).reset_index()
contentdf.shape

(41772, 2)

DROP movies with low numbers of keywords

In [15]:
contentdf = contentdf[contentdf['words'].str.count(' ') >= 4]
contentdf.shape

(33393, 2)

Use TfIdfVectorizer instead of CountVectorizer, a simple words frequecies counter because it gives less importance to "high presence words" like the genre of movies.

Use Cosine similarity to obtain the similarity matrix between movies 

In [16]:
count_words = TfidfVectorizer(analyzer='word', stop_words='english')
count_words_matrix = count_words.fit_transform(contentdf['words'])

In [17]:
similarity_cosine = cosine_similarity(count_words_matrix, count_words_matrix)

In [18]:
contentdf = contentdf.reset_index()
def get_recommendations(title, number_recommendations):
    index_movie = contentdf[contentdf['title'] == title].index.item()
    similarity_scores = list(enumerate(similarity_cosine[index_movie]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:number_recommendations+1]
    movie_indices = [ x[0] for x in similarity_scores if x[0] > 0 ]
    return contentdf['title'].iloc[movie_indices]

In [35]:
get_recommendations('Titanic', 15)

10026               Ghosts of the Abyss
888                 A Night to Remember
15062                        Love Story
30612                          Titanica
2130                 Angels and Insects
17578                 Nobody's Children
32253                           Welfare
14606                            Lilies
17572                   Nobody Loves Me
12306                           In Time
11955                    I Live My Life
11756                         Hubble 3D
20049    Requiem for the American Dream
10749            Hands Across the Table
33094        You Can't Take It With You
Name: title, dtype: object

# **Collaborative Filtering** - KNN

In [20]:
users = pd.read_csv("/kaggle/input/the-movies-dataset/ratings_small.csv")[['userId', 'movieId', 'rating']] #Change with ratings.csv
users.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


Merge id (movies_metadata.csv) and tmdbId (links.csv)

In [21]:
links = pd.read_csv("/kaggle/input/the-movies-dataset/links_small.csv")[['movieId', 'tmdbId']]
movies_temp = movies_metadata[['id', 'title']].drop([19730, 29503, 35587])
movies_temp['id'] = movies_temp['id'].astype(int)

In [22]:
links = users.merge(links, on="movieId").merge(movies_temp, left_on="tmdbId", right_on="id")[['movieId','title']].drop_duplicates().dropna()
links.head()

,movieId,title
0,31,Dangerous Minds
42,1029,Dumbo
84,1061,Sleepers
117,1129,Escape from New York
165,1172,Cinema Paradiso


In [23]:
matrix_ratings = users.reset_index().pivot(index='movieId', columns='userId', values='rating').fillna(0)
matrix_ratings

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
knn.fit(matrix_ratings)

NearestNeighbors(algorithm='brute', metric='cosine')

In [25]:
def get_collab_movies(title, number_recommendations):
    index_movie = links[links['title'] == title]['movieId'].iloc[0]
    distances, indices = knn.kneighbors(matrix_ratings[matrix_ratings.index == index_movie], n_neighbors = number_recommendations+1)
    ind_distances = list(zip(indices.squeeze().tolist(), distances.squeeze().tolist()))[1:]
    movies_id = [matrix_ratings.iloc[x[0]].name for x in ind_distances]
    return [ links[links['movieId'] == i]['title'].item() for i in movies_id if len(links[links['movieId'] == i]['title']) > 0 ]

In [26]:
get_collab_movies('Titanic', 15)

['Men in Black',
 'The Sixth Sense',
 'The Truman Show',
 'Forrest Gump',
 'Saving Private Ryan',
 'Jerry Maguire',
 "There's Something About Mary",
 'Good Will Hunting',
 'Back to the Future',
 'The Matrix',
 'Shrek',
 "Ocean's Eleven",
 'Spider-Man',
 'Pretty Woman',
 'E.T. the Extra-Terrestrial']

# **Hybrid**

In [27]:
def score(title, number_recommendations, df):
    sum = (number_recommendations - df[df['title'] == title].index[0])
    if title in df['collaborative'].values: sum = sum + (number_recommendations - (df[df['collaborative'] == title].index[0]))
    return sum

In [28]:
def hybrid_recommender(title, number_recommendations):
    content = get_recommendations(title, number_recommendations).reset_index(drop=True)
    collaborative = pd.Series(get_collab_movies(title, number_recommendations), name='collaborative').reset_index(drop=True)
    hybrid = pd.concat([content,collaborative], axis=1)
    hybrid['score'] = hybrid['title'].apply(lambda x: score(x, number_recommendations, hybrid))
    return hybrid.sort_values(by=['score'], ascending=False)[:15]['title']

In [38]:
hybrid_recommender('The Matrix', 1000)

97                        Sophie's Choice
92                          Random Hearts
101    Butch Cassidy and the Sundance Kid
120                             She-Devil
15               Three Days of the Condor
187                             The Sting
133                       Ordinary People
183                     Kramer vs. Kramer
62                         One True Thing
149               A River Runs Through It
119                              Sneakers
60                              Heartburn
305                 Wide Eyed and Legless
4                      Mr. Saturday Night
81                               The Firm
Name: title, dtype: object